# Python Decorators  - what they are and how they work

---

__Elliott Forney - 2020__

Decorators are a useful and flexible construct in python that can be used in may ways.  In this notebook, we will first explore some common ways that python decorators can be used.  We will then examine how decoators work and see that they are really just syntactic sugar around wrapper functions.  Using this insight, we'll see how to write custom function decorators and, finally, we will also see how to build custom decorators for classes.

## Common decorators

A number of common decorators have found there way into the standard Python API.  A good place to start looking at decorators is to examine this standard functionality and see how it is generally used.

### `@staticmethod`

The `staticmethod` decorator allows methods to be defined on classes that are not bound to an instance of the class and do not reference the `self` variable that we typically see with member functions.  This is similar to the `static` keyword that can be placed on methods in Java and C++.

In the example below, we create a very simple class that represents and "artificial agent."  This class can be instantiated and interacted with using some (very naive) natural language requests.

One of the tasks that the agent can perform is to compute the value of the irrational number $\phi$, i.e., the "golden ratio."

Note, however, that computing $\phi$ doesn't require any class instance information and can be implemented as an isolated function.  In other words, it doesn't need to reference `self`, which is a good rule of thumb for when to use the `@staticmethod` decorator.

We could simply place the `compute_phi` function outside of the class and make it free-standing.  Since it is generally related to our `ArtificialAgent` class, however, it may make more sense to simply keep this free function inside of the class and declare it a static method.

<font color="maroon">__sidenote:__ Python classes can be "callable," just like functions.  All you have to do is implement the `__call__` method.</font>

In [2]:
class ArtificialAgent:
    '''Artificial agent that can interact using a super
    naive natural language interface.
    '''
    def __init__(self, name='HAL9000'):
        '''Create a new agent with a name.'''
        self.name = name
        
    def __call__(self, message):
        '''Calling an instance directly interfaces with
        the agent using natural language.
        '''
        message = message.lower()

        if message.startswith('hello'):
            return f'Hello!  My name is {self.name}.'
        
        if message.startswith('how are you'):
            return 'I am well.  How are you Dave?'
        
        if message.endswith('phi.'):
            return self.compute_phi()
        
        return 'Sorry, I do not understand.'
    
    @staticmethod
    def compute_phi():
        '''Compute the value of $\phi$ (the golden ratio).
        This does not need to reference `self`, so it can be
        a `staticmethod`.
        '''
        table = [1, 1]
        for i in range(2, 1000):
            table.append(table[i-1] + table[i-2])
            
        return table[-1] / table[-2]

Next, we can create an instance of our artificial agent and and ask it to compute $\phi$.  Internally, this calls the static `compute_phi` method.

In [43]:
agent = ArtificialAgent()

In [44]:
agent('hello')

'Hello!  My name is HAL9000.'

In [45]:
agent('How are you doing?')

'I am well.  How are you Dave?'

In [46]:
agent('HAL, please compute the value of Phi.')

1.618033988749895

Notice, however, that __we don't need to instantiate the class in order to call `compute_phi`__!

Since static methods do not reference self, they can be called directly on the uninstantiated `class`.

In [47]:
ArtificialAgent.compute_phi()

1.618033988749895

The `@staticmethod` can be super valueable for organizing your code in a way the methods related to a class remain in the class, even if they do not require information about a specific instance.

### `@classmethod`

The `@classmethod` decorator is another useful decorator found in the standard Python API that allows us to create methods that are passed an argument called `cls` that is the actual class (__not__ instance/`self`) as the first argument.  Class methods then return an instance of `cls`.

The most popular use of this decorator is to create methods that return new instances of a class, in addition to the classes primary "constructor," i.e., in addition to the `__init__` method.  In order to see why this is so useful, recall that Python does not allow method overloading, i.e., you can't have multiple function signatures with the same name but different arguments.  This means that we cannot overload the constructor in order to create class instances in different ways!!

So, if we need to have multiple methods for creating class instances, how can we do that?  Hooray for class methods!  The general approach for having multiple constructors in Python is to first define an `__init__` method that takes the simplest, most basic arguments required to instantiate the class.

Then, we can define additional methods with the `@classmethod` decorator that process different inputs and then call the primary constructor of the class.

In order to see how this works, consider the example below where we have a class called `ServerInfo` that tracks an ip address and port for a generic server.  The default `__init__` takes these most basic arguments, a string `ip` and `port`.  But what if we want to construct a server from a hostname?  The answer is to create a class method that takes `hostname` and `port`, performs the hostname resolution to get `ip` and then calls `__init__` with `ip` and `port`.

In [10]:
import socket, ipaddress

In [52]:
class ServerInfo:
    '''Basic information about a server on the internet.'''
    def __init__(self, ip, port):
        '''Construct server information from a string `ip` address
        and an integer `port` number.'''
        self.ip = str(ipaddress.ip_address(ip))
        self.port = int(port)

    @classmethod
    def from_hostname(cls, host, port):
        '''Construct server information from a string `hostname`
        and an integer `port` number.  This method resolves
        `hostname` into an ip address and then returns an instance.
        '''
        ip = socket.gethostbyname(host)
        return cls(ip, port)
        
    def __repr__(self):
        '''String representation of our server information.'''
        return f'{self.ip}:{self.port}'

Of course, we can create an instance of `ServerInfo` via the `__init__` method by simply calling the class with the `ip` address and `port` number arguments.

In [49]:
ServerInfo('127.0.0.1', 8080)

127.0.0.1:8080

If we have a `hostname` instead of an `ip` address, we can instead call the `from_hostname` class method on the `ServerInfo` class.  This method takes the class as an argument, looks up the ip address, and then returns an instance of the class.

In [50]:
ServerInfo.from_hostname('localhost', 8080)

127.0.0.1:8080

In [51]:
ServerInfo.from_hostname('www.google.com', 8080)

172.217.1.196:8080

But why do we need the `@classmethod` decorator in order to do this?  Couldn't we just create a `@staticmethod` that returns an instance of `ServerInfo`?

Well, we could, but that wouldn't play nicely with inheritance.  In other words, if `from_hostname` returned `ServerInfo(ip, port)` then if we create a subclass called, say, `CustomServerInfo`, then the static method would, incorrectly, return an instance of the parent class.  The `@classmethod` decorator hides the details of finding the appropriate class and simply passes it as the `cls` argument for consumption.

In order to see this, consider the example where we create a subclass of `ServerInfo` called `CustomServerInfo`.  This class behaves the same as it's parent but adds a method that checks if the ip/port combo is listening for connections

<font color="maroon">__sidenote:__ The `socket` class provides a context manager, so you can use the `with` keyword instead of `try`/`except`/`finally`/`close`.

In [53]:
class CustomServerInfo(ServerInfo):
    def check_open(self):
        '''Return `True` if the server is currently listening
        for new connections and `False` otherwise.
        '''
        with socket.socket(socket.AF_INET) as sock:
            sock.settimeout(1.0)
            return not bool(sock.connect_ex((self.ip, self.port)))

Notice that if we call the `from_hostname` class method, we get an instance of `CustomServerInfo`!!  Class methods play well with inheritance.

In [54]:
info = CustomServerInfo.from_hostname('www.google.com', 443)

In [55]:
type(info)

__main__.CustomServerInfo

And, of course, we are able to leverage the functionality provided by the child class.

In [56]:
info.check_open()

True

In [57]:
info = CustomServerInfo.from_hostname('www.google.com', 8080)

In [58]:
info.check_open()

False

### `@property`

The `@property` decorator is another extremely useful piece of functionality provided by the Python API.  This decorator allows us to create functions that *appear* to be attributes (member variables) to the user.

Properties can be used in many ways, but their primary use is to allow developers to avoid writting getters and setters without fear of possibly breaking the API in the future.

In many object-oriented languages, e.g., Java and C++, it is recomended as a best practice to write getter and setter methods, a.k.a. accessor and mutator methods, for all attributes of a class unless the developer is *absolutely certain that there will never be a need in the future to compuate it dynamically or lazily*.  Since this is a "big if," you often see APIs in these language with lots of `get_this` and `set_that` methods that do nothing more than set or return a member variable.

In Python, properties allow us to not be concerned about possibly breaking the API in the future, by allowing class attributes to be replaced by methods in a way that is transparent to the caller.  In order to see this, consider the example where we have a `Circle` class that is constructed via the `radius` of a circle and where the caller will need to be able to get the area of the corresponding circle.

In a simple implementation, it's easy enough to simply compute the area of the circle when it's constructed and save it as an attribute.

In [59]:
class Circle1:
    '''A circle with an attribute called `area` that is
    computed when the circle is instantiated.
    '''
    def __init__(self, radius):
        self.radius = radius
        self.area = 3.14159 * self.radius**2

In [60]:
c1 = Circle1(12.0)
c1.area

452.38896

But what if, someday, we are profiling our code and realize that (a) we are creating many millions of circles and (b) we only access the area for about 1% of those instances.  We could improve computation tremendously if we simply compute `area` when it is requested instead of during the construction of *all* circles.

Now, that's great, but if we need to change the attribute `circle.area` to a method called `circle.get_area()`, then we will break all of the code that uses the `Circle` class.  In a large project or one that is used by other downstream code bases, this may be a tremendous undertaking that makes many developers and, potentially customers, unhappy.  In order to avoid this situation in the first place, one might simply start with `.get_area()` instead of making it an attribute to start with.

In Python, however, this does not need to be a concern.  Instead, we can simply decorate a method called `area()` with `@property` and voila!  We can now compute area dynamically and it still appears to be an attribute to the caller.  The API does not break.

In [32]:
class Circle2:
    '''A circle with a property called `area`.  Now,
    the area is only computed when the property is
    accessed, which may save computation time if it
    is rare to access the area of the circle.
    '''
    def __init__(self, radius):
        self.radius = radius
        
    @property
    def area(self):
        return 3.14159 * self.radius**2

In [33]:
c2 = Circle2(12.0)
c2.area

452.38896

What if it turns out though that the `area` property is often accessed many times for each instance of a circle, even if it is only for 1% of the total circles?  This suggests that we should lazily compute the area, i.e., only do the computation if it is requested and also don't compute it multiple times if it requested repeatedly.

As it turns out, that can also be easily done with properties.  We simply need to define a hidden attribute, say `_area` that is `None` until it is computed the first time.  From then on, the property can simply return already computed value.  This approach can be extremely valueable for performance tuning in some circumstances.

In [61]:
class Circle3:
    def __init__(self, radius):
        self.radius = radius
        self._area = None
        
    @property
    def area(self):
        if self._area is None:
            self._area = 3.14159 * self.radius**2

        return self._area

In [62]:
c3 = Circle3(12.0)

In [63]:
c3.area

452.38896

In [64]:
c3.area

452.38896

In [65]:
%%timeit
c1.area

48.5 ns ± 3.78 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [66]:
%%timeit
c2.area

204 ns ± 14.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [67]:
%%timeit
c3.area

179 ns ± 5.81 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


## Custom function decorators

So, yea, decorators are great for concisely solving some kinds of problems!  But how do they work?  and, more importantly, how do I create my own decorators if I want to?  It turns out that it's really fairly easy and in this section we'll see how.

### `@logged`

Let's start out with a simple yet useful example of logging function calls.  Say that we want to be able to simply add a decorator to a function that causes it to print a log message to the console.  This can be useful for debugging and logging when we don't necessarily have or want to dig into stack traces.

Let's start with the legendary hello world function and see if we can add a log message to it.

In [69]:
def hello_world(value):
    print(f'Hello World! {value}')

In [70]:
hello_world(42)

Hello World! 42


In [71]:
def logged(func):
    def wrapper(*args, **kwargs):
        print(f'=== calling function `{func.__name__}` ===')
        return func(*args, **kwargs)
    return wrapper

In [72]:
hello_world = logged(hello_world)
hello_world(99)

=== calling function `hello_world` ===
Hello World! 99


In [73]:
@logged
def hello_world(value):
    print(f'Hello World! {value}')

In [74]:
hello_world(42)

=== calling function `hello_world` ===
Hello World! 42


### `@timed`

In [79]:
import functools

In [80]:
import time

def timed(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        total = time.time() - start
        print(f'=== calling `{func.__name__}` took {total:.4} seconds ===')
        return result
    return wrapper

In [90]:
@timed
def product(values):
    accum = 0.0
    for value in values:
        accum *= value
    return accum

#product = timed(product)

In [91]:
product(range(1, 1024))

=== calling `product` took 7.343e-05 seconds ===


0.0

## Passing arguments to a decorator

### `@log_message`

In [93]:
def log_message(message):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(message)
            return func(*args, **kwargs)
        return wrapper
    return decorator

In [98]:
@log_message('Calling a function that says hello!')
def say_hello(rep=1):
    '''Print the word "hello" to standard out `rep` times.'''
    for _ in range(rep):
        print('hello')

In [99]:
say_hello(3)

Calling a function that says hello!
hello
hello
hello


In [100]:
@log_message("computing the square root with newton's method...")
def sqrt(v, precision=1.0e-5, eps=1.0e-10):
    x0, x1 = float('inf'), 0.0
    while abs(x1 - x0) > precision:
        x0, x1 = x1, x1 - (x1**2 - v) / max((2.0 * x1), eps)
    return x1

In [101]:
sqrt(20)

computing the square root with newton's method...


4.47213595499958

### `@suppress_exception`

In [102]:
def suppress_exception(exc):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            try:
                return func(*args, **kwargs)
            except exc as ex:
                print(f'ignoring `{exc.__name__}`')
                return None
        return wrapper
    return decorator

In [107]:
@suppress_exception(RuntimeError)
def raise_runtime_error():
    raise RuntimeError('A runtime error happened!')

In [108]:
raise_runtime_error()

ignoring `RuntimeError`


## Chaining Decorators

In [111]:
@timed
@log_message('Custom message')
@suppress_exception(ValueError)
def add(x1, x2):
    '''Add to floats, `x1` and `x2`.'''
    if not isinstance(x1, float) or not isinstance(x2, float):
        raise ValueError('argument was not a float!')
    return x1 + x2

In [112]:
add(1.1, 2.2)

Custom message
=== calling `add` took 3.982e-05 seconds ===


3.3000000000000003

In [113]:
add('hello', 3.14159)

Custom message
ignoring `ValueError`
=== calling `add` took 0.000114 seconds ===


In [114]:
help(add)

Help on function add in module __main__:

add(x1, x2)
    Add to floats, `x1` and `x2`.



## Custom class decorators

### `@registered`

In [118]:
import uuid

class_registry = {}

In [119]:
def registered(cls):
    cls.identifier = str(uuid.uuid1())
    class_registry[cls.identifier] = cls
    return cls

In [120]:
@registered
class Sphere:
    pass
    
@registered
class Dodecahedron:
    pass

@registered
class Scutoid:
    pass

In [121]:
class_registry

{'0846d8fe-6bea-11eb-adaf-acde48001122': __main__.Sphere,
 '0846e09c-6bea-11eb-adaf-acde48001122': __main__.Dodecahedron,
 '0846e394-6bea-11eb-adaf-acde48001122': __main__.Scutoid}

### `@stylize`

<font color="maroon">__sidenote__: the wrapper functions below create a "lexical closure" around the variables `font_weight` and `style`.

In [122]:
def stylize(color, bold=False, font='monospace'):
    font_weight = 'bold' if bold else 'normal'
    style = f'color: {color}; font-weight: {font_weight}; font-family: {font}'
    
    def decorator(cls):
        def new_repr(self):
            return f'<pre style="{style}">' + self.__repr__() + '</pre>'
        
        cls._repr_html_ = new_repr
        
        return cls
    return decorator

In [137]:
@stylize(color='red', bold=False, font='serif')
class Square:
    def __init__(self, width):
        self.width = width
        
    def __repr__(self):
        return '\n'.join(['X'*self.width,]*self.width)

In [138]:
square = Square(2)
square

XX
XX

In [139]:
square = Square(4)
square

XXXX
XXXX
XXXX
XXXX

In [140]:
import numpy as np

@stylize(color='purple', font='serif')
class RandomData:
    def __init__(self, n):
        self.data = np.random.random(n)
        
    def __repr__(self):
        return repr(self.data)

In [141]:
data = RandomData(100)
data

array([0.0922, 0.5469, 0.9968, 0.0966, 0.9563, 0.5108, 0.4907, 0.1776,
       0.8905, 0.9962, 0.212 , 0.1025, 0.4567, 0.4712, 0.6405, 0.7476,
       0.5465, 0.5829, 0.6325, 0.7915, 0.6361, 0.2429, 0.415 , 0.9988,
       0.445 , 0.7444, 0.3916, 0.2252, 0.7578, 0.1101, 0.9083, 0.572 ,
       0.1022, 0.0092, 0.1414, 0.3133, 0.5655, 0.8741, 0.457 , 0.959 ,
       0.4543, 0.6013, 0.0043, 0.2282, 0.6686, 0.1844, 0.1517, 0.0295,
       0.2018, 0.2561, 0.4632, 0.2562, 0.2519, 0.6873, 0.9197, 0.6869,
       0.1068, 0.7443, 0.6663, 0.4446, 0.3142, 0.2793, 0.3039, 0.8691,
       0.4356, 0.2385, 0.8402, 0.4476, 0.819 , 0.6823, 0.6851, 0.8339,
       0.1813, 0.2139, 0.085 , 0.8721, 0.2792, 0.3422, 0.3052, 0.7493,
       0.2259, 0.3274, 0.5593, 0.4811, 0.6952, 0.5523, 0.8638, 0.7961,
       0.7409, 0.7784, 0.9191, 0.9219, 0.755 , 0.5668, 0.0261, 0.0817,
       0.6316, 0.4184, 0.9768, 0.6782])

## Summary

* Common decorators like `classmethod` and `staticmethod` and `property` are extremely useful for writing clear and extensible python code.  Get to know these decorators.
* Decorators are really just syntactic sugar around wrapper functions that are called at import time!  This makes it easy to write custom decorators using functional-style programming, including lexical closures.
* Use `functools.wrapps` to preserve docstrings when writing custom decorators.
* Decorators can take arguments by using *another* layer of wrapper functions.  The outer function actually builds the decorator function using it's arguments.
* Decorators can also be chained together.  Applying multiple decorators in a row simply wraps the function returned by the previous decorator.
* Classes can leverage decorators too!  When a decorator is placed on a class, the wrapper function takes the `class` (not the instance) and can modify it at import time.